In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class RegHead(nn.Module):
    def __init__(self,in_channels, S, B, dropout=0.0):
        super().__init__()
        self.conv0 = nn.Conv2d(in_channels, 1024, 3, padding=1)
        self.conv1 = nn.Conv2d(1024, 1024, 3, padding=1)
        self.conv2 = nn.Conv2d(1024, 1024, 3, padding=1)
        self.conv3 = nn.Conv2d(1024, 1024, 3, padding=1)

        self.l0 = nn.Linear((1024 * S * S), 4096)
        self.drop = nn.Dropout(dropout)
        self.act = nn.LeakyReLU(0.1)
        self.l1 = nn.Linear(4096, (S * S * (B * 5)))

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.flatten(start_dim=1)
        x = self.l0(x)
        x = self.drop(x)
        x = self.act(x)
        x = self.l1(x)
        return x




class FaceDetector(nn.Module):
    def __init__(self, S=7, B=9):
        super(FaceDetector, self).__init__()

        backbone = models.resnet50(pretrained=True)
        self.backbone = torch.nn.Sequential(*(list(backbone.children())[:-2]))
        
        self.regression_head = RegHead(2048, S, B)
    
    def forward(self, x):
        features = self.backbone(x)
        out = self.regression_head(features)
        return out

In [18]:
x = torch.rand((32,3,224,224))
x.shape

torch.Size([32, 3, 224, 224])

In [19]:
model= FaceDetector()

/home/kuba/.virenv/base/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kuba/.virenv/base/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
model(x).shape


torch.Size([32, 50176])


torch.Size([32, 2205])